In [ ]:
# PASSO 1 - EXTRAIR COMMITS. 
##Pequisa em cada projeto seus commits que tenha 'refactor' e salva em JSON

import os
import json
from git import Repo

# Caminho para a pasta com os projetos do GitHub
# path = './../1_PROJETOS/projects/apache'
# path = './../1_PROJETOS/projects/eclipse'
path = './../1_PROJETOS/projects/random'

# Dicionário para armazenar os commits
project_commits = {}
# project_commits = {}

# Percorre todos os diretórios na pasta
for project in os.listdir(path):
    print(project)
    project_path = os.path.join(path, project)
    
    # Verifica se é um diretório
    if os.path.isdir(project_path):
        repo = Repo(project_path)
        
        # Lista para armazenar os commits
        commits = []
        
        # Percorre todos os commits do repositório
        for commit in repo.iter_commits():
            # Verifica se a palavra 'refactor' está na mensagem do commit
            if 'refactor' in commit.message:
                commits.append({'hash': str(commit), 'message': commit.message})
        
        # Adiciona o projeto com os commits à lista de projetos
        # projects.append({project: commits})
        project_commits[project] = commits

# Cria o arquivo JSON
with open('commits.json', 'w') as f:
    json.dump(project_commits, f)


In [ ]:
# PASSO 2 - EXTRAIR REFATORAÇÕES DE CADA COMMIT
import json
import subprocess
import os
import datetime
import pickle

# Caminho para a pasta onde os projetos estão localizados
projects_folder_path = "./../1_PROJETOS/projects/random"

# Ler o arquivo JSON
with open("./commits.json", "r") as json_file:
    data = json.load(json_file)

# Lista para armazenar os resultados
results = []

for project_name, project_data in data.items():
    # Construir o caminho para o repositório
    git_repo_path = os.path.join(projects_folder_path, project_name)

    # Verificar se a pasta do projeto existe
    if not os.path.exists(git_repo_path):
        print(f"A pasta do projeto {project_name} não existe")
        continue

    # Lista para armazenar as refatorações do projeto
    refactorings = []

    # Iterar sobre cada commit no array de commits do projeto
    for commit_data in project_data:
        hora_atual = datetime.datetime.now().time()
        print(f'{hora_atual} - Analisando commit {commit_data["hash"]} do projeto {project_name}')

        # Executar o Refactoring Miner na linha de comando
        try:
            command = f"refactoringminer -c {git_repo_path} {commit_data['hash']} -json temp.json"
            result = subprocess.run(command, shell=True, capture_output=True, text=True)
        except Exception as e:
            print(f"Erro ao executar o comando para o commit {commit_data['hash']} - {e}")
            continue
        # Verificar se o comando foi executado com sucesso
        if result.returncode == 0:
            # Adicionar o resultado à lista de refatorações
            refactorings.append(result.stdout)
            try:
                with open ("temp.json", "r", encoding='utf-8') as json_file:
                    data = json.load(json_file)
                    refactorings.append(data)
            except Exception as e:
                print(f"Erro ao carregar o arquivo JSON {e}")
        else:
            print(f"Erro ao executar o comando para o commit {commit_data['hash']}")

    # Adicionar o projeto com as refatorações à lista de resultados
    results.append({'project': project_name, 'refactorings': refactorings})


    # Salvar 'results' usando pickle antes de gerar o JSON final
    with open('results.pkl', 'wb') as f:
        pickle.dump(results, f)

# Gerar o JSON final
json_final = json.dumps(results)

# Salvar o JSON final em um arquivo
with open('final.json', 'w') as json_file:
    json_file.write(json_final)


In [ ]:
#EXTRAIR CRIAR SUMMARY COMMIT-OPERATIONS
import json

# datasetName='apache'
# datasetName='eclipse'
datasetName='random'
x = open(f'./{datasetName}_refactorings_final.json', 'r')
myJSON = json.load(x)

commitOperations=[]
#Iterar sobre os projetos
for i,projeto in enumerate(myJSON):
    # print(f'{i}/{len(myJSON)}')
    # Acessar as refatorações
    for refatoracao in projeto["refactorings"]:
        # Verificar se a refatoração é um dicionário e contém a chave 'commits'
        if isinstance(refatoracao, dict) and "commits" in refatoracao:
            # Acessar os commits com refatorações
            for commitRow in refatoracao["commits"]:
                commit = commitRow["sha1"]
                operations =[]
                for ref in commitRow['refactorings']:
                    operations.append(ref['type'])
                commitOperations.append({'commit':commit,'operations':operations})

with open(f'{datasetName}_commit_operations.json', 'w') as f:
    json.dump(commitOperations, f)
